In [2]:
import json

# read json file
json_str = open('api.waqi.info.json').read()
data = json.loads(json_str)

In [21]:
data

{'status': 'ok',
 'data': [{'lat': 3.177084,
   'lon': 113.041091,
   'uid': 2607,
   'aqi': '42',
   'station': {'name': 'Bintulu, Sarawak, Malaysia',
    'time': '2025-09-29T22:00:00+09:00'}},
  {'lat': 4.5831986,
   'lon': 114.2212174,
   'uid': 8116,
   'aqi': '13',
   'station': {'name': 'Kuala Belait, Brunei',
    'time': '2025-09-29T17:00:00+09:00'}},
  {'lat': 2.926645,
   'lon': 103.419198,
   'uid': 9496,
   'aqi': '7',
   'station': {'name': 'Rompin, Pahang, Malaysia',
    'time': '2025-09-29T22:00:00+09:00'}},
  {'lat': 4.200344,
   'lon': 100.663358,
   'uid': 2597,
   'aqi': '29',
   'station': {'name': 'Seri Manjung, Perak, Malaysia',
    'time': '2025-09-29T22:00:00+09:00'}},
  {'lat': 5.401688,
   'lon': 100.58968,
   'uid': 9488,
   'aqi': '45',
   'station': {'name': 'Kulim Hi-Tech, Kedah, Malaysia',
    'time': '2025-09-29T22:00:00+09:00'}},
  {'lat': 2.014498,
   'lon': 112.92736,
   'uid': 2609,
   'aqi': '39',
   'station': {'name': 'Kapit, Sarawak, Malaysia',
  

In [40]:
import pandas as pd

malaysia_df = pd.DataFrame(columns=['uid', 'aqi', 'name', 'time', 'lat', 'lon'])

for entry in data['data']:
    if entry['station']['name'].find('Malaysia') != -1:
        series_entry = [entry['uid'], entry['aqi'], entry['station']['name'], entry['station']['time'], entry['lat'], entry['lon']]
        malaysia_df.loc[len(malaysia_df)] = series_entry

In [41]:
#Get value counts of list lengths
malaysia_df['name'].str.split(',').str.len().value_counts()

name
3    56
4     9
Name: count, dtype: int64

In [42]:
# Create a mask for rows that don't have exactly 3 comma-separated parts
mask = malaysia_df['name'].str.split(',').str.len() != 3

# Apply regex replacement only to those rows
malaysia_df.loc[mask, 'name'] = malaysia_df.loc[mask, 'name'].str.replace(r'^[^,]*,\s*', '', regex=True)

In [43]:
#Get value counts of list lengths
malaysia_df['name'].str.split(',').str.len().value_counts()

name
3    65
Name: count, dtype: int64

In [44]:
# split name into district, state, and country
malaysia_df[['district', 'state', 'country']] = malaysia_df['name'].str.split(',', expand=True)

# clean up whitespace
malaysia_df['district'] = malaysia_df['district'].str.strip()
malaysia_df['state'] = malaysia_df['state'].str.strip()
malaysia_df['country'] = malaysia_df['country'].str.strip()

# remove name
malaysia_df = malaysia_df.drop(columns=['name'])

In [45]:
malaysia_df

,uid,aqi,time,lat,lon,district,state,country
0,2607,42,2025-09-29T22:00:00+09:00,3.177084,113.041091,Bintulu,Sarawak,Malaysia
1,9496,7,2025-09-29T22:00:00+09:00,2.926645,103.419198,Rompin,Pahang,Malaysia
2,2597,29,2025-09-29T22:00:00+09:00,4.200344,100.663358,Seri Manjung,Perak,Malaysia
3,9488,45,2025-09-29T22:00:00+09:00,5.401688,100.589680,Kulim Hi-Tech,Kedah,Malaysia
4,2609,39,2025-09-29T22:00:00+09:00,2.014498,112.927360,Kapit,Sarawak,Malaysia
...,...,...,...,...,...,...,...,...
60,10485,51,2025-09-29T22:00:00+09:00,2.914816,101.690050,Putrajaya,W.p. Putrajaya,Malaysia
61,2603,43,2025-09-29T22:00:00+09:00,5.339317,116.163658,Keningau,Sabah,Malaysia
62,2591,30,2025-09-29T22:00:00+09:00,3.960644,103.382158,Kuantan,Pahang,Malaysia
63,2622,56,2025-09-29T22:00:00+09:00,4.262121,103.425778,Kemaman,Terengganu,Malaysia


In [46]:
# save as csv
malaysia_df.to_csv('malaysia_aqi.csv', index=False)